# faker 라이브러리 설치
- https://faker.readthedocs.io/en/master/index.html

In [128]:
!pip install Faker

# 라이브러리 임포트

In [129]:
from faker import Faker

# 성별 무관한 이름 출력

In [130]:
fake = Faker('ko_KR')
Faker.seed(4321) # seed 지정하면 고정값이 나옴.

for _ in range(10):
    print(fake.name_nonbinary())

이도윤
이지후
김광수
조승민
이하은
김민재
고상훈
류윤서
김예준
김주원


# 성별 무관한 이름(unique) 출력

In [131]:
fake = Faker('ko_KR')
names = [fake.unique.name_nonbinary() for i in range(500)]
assert len(set(names)) == len(names)

## 해설

Python 코드에서 **assert**는 **디버깅 도구**로 사용됩니다. 특정 조건이 **참(True)**인지 확인하며, 조건이 **거짓(False)**일 경우 AssertionError를 발생시킵니다. 이를 통해 코드의 논리가 의도한 대로 동작하는지 확인할 수 있습니다.

### 코드 동작
1. Faker('ko_KR'):
   - 한국어 기반으로 가짜 데이터를 생성하는 Faker 객체를 생성.
2. [fake.unique.name_nonbinary() for i in range(500)]:
   - fake.unique는 중복되지 않는 데이터를 생성하는 데 사용됩니다.
   - 500개의 중성 이름을 리스트에 생성.
3. len(set(names)) == len(names):
   - set(names): 리스트 names에서 중복을 제거한 집합(Set)을 생성.
   - len(set(names)) == len(names): 리스트의 길이와 중복을 제거한 집합의 길이가 동일한지 확인.
     - 동일하다면 names 리스트에 중복된 값이 없다는 의미.
     - 다르다면 중복된 값이 있다는 뜻입니다.
5. assert:
   - 이 조건을 확인합니다. 조건이 참이면 아무 동작도 하지 않고 코드가 계속 실행됩니다.
   - 하지만 조건이 거짓이면 AssertionError가 발생하여 실행을 멈춥니다.


# 만들어야 할 테이블 정의

## tb_user (사이트 회원 정보) - 1000명

- "user_id" VARCHAR(50) **'사용자 아이디(이메일)'**]
    <br>: **겹치면 안 됨**.
- "user_pw" VARCHAR(50) [not null, note: **사용자 비밀번호'**]
    <br>: 그냥 통일(나중에 암호화) : **test11111111**
- "user_name" VARCHAR(50) [not null, note: '**사용자 이름**]
- "user_phone" VARCHAR(20) [not null, note: '**사용자 전화번호**]
- "user_profile_img" VARCHAR(1000) [not null, note: **사용자 프로필 사진**.']
    <br>: **'일단 임의로 "/img/"**
- "user_role" VARCHAR(10) [not null, note: '**사용자 역할**']
    <br>: **멘토링 받을 때 의도치 않게 들어간 컬럼. 일단 "member"로 고정, 삭제...?**
-  "joined_at" TIMESTAMP [not null, default: `CURRENT_TIMESTAMP`, note: **생성 일자**']
    <br> : **FROM 지금으로부터 1년 전 TO 지금으로부터 6개월 전.**

In [132]:
from faker import Faker
import random
import pandas as pd

# 시드 설정
random.seed(42)
fake = Faker('ko_KR')
Faker.seed(42)

def generate_korean_phone_number(existing_phones):
    """Generate a unique Korean phone number in the format 010-xxxx-xxxx"""
    while True:
        first = "010"
        second = str(random.randint(1000, 9999))  # 4자리 숫자
        third = str(random.randint(1000, 9999))  # 4자리 숫자
        phone = f"{first}-{second}-{third}"
        if phone not in existing_phones:
            existing_phones.add(phone)
            return phone

def generate_user_data(num_users):
    users = []
    korean_domains = ["naver.com", "daum.net", "gmail.com", "kakao.com", "hanmail.net"]  # 한국 도메인 리스트
    existing_emails = set()  # 유니크 이메일 저장
    existing_phones = set()  # 유니크 전화번호 저장
    
    for _ in range(num_users):
        # 유니크 이메일 생성
        while True:
            email_local = fake.email().split('@')[0]  # '@' 앞부분
            email_domain = random.choice(korean_domains)  # 랜덤 한국 도메인 선택
            user_email = f"{email_local}@{email_domain}"  # 이메일 재조합
            if user_email not in existing_emails:
                existing_emails.add(user_email)
                break
        
        # 유니크 전화번호 생성
        user_phone = generate_korean_phone_number(existing_phones)
        
        joined_date = fake.date_between(start_date="-1y", end_date="-6m")
        
        user = {
            "user_id": user_email,
            "user_pw": "test12341234",  # 예시용 고정값
            "user_name": fake.unique.name_nonbinary(),
            "user_phone": user_phone,
            "user_profile_img": "/img/img1.png",
            "user_role": "member",
            "joined_at": joined_date.strftime('%Y-%m-%d %H:%M:%S'),
        }
        users.append(user)
    return users

# 1000명 데이터 생성
df_users = pd.DataFrame(generate_user_data(1000))

# 결과 확인
df_users.head()

,user_id,user_pw,user_name,user_phone,user_profile_img,user_role,joined_at
0,gimyejun@naver.com,test12341234,장춘자,010-1409-5506,/img/img1.png,member,2024-02-18 00:00:00
1,coedoyun@daum.net,test12341234,조영호,010-4657-3286,/img/img1.png,member,2024-03-19 00:00:00
2,lgim@naver.com,test12341234,권민지,010-9935-2424,/img/img1.png,member,2024-11-04 00:00:00
3,ji@hanmail.net,test12341234,김동현,010-7912-1520,/img/img1.png,member,2024-10-17 00:00:00
4,sumin07@naver.com,test12341234,이유진,010-2535-4582,/img/img1.png,member,2024-02-12 00:00:00


In [133]:
# CSV 파일로 저장
csv_file_name = "tb_user.csv"  # 저장할 파일 이름
df_users.to_csv(csv_file_name, index=False, encoding='utf-8-sig')  # UTF-8 인코딩으로 저장 (한글 지원)

print(f"파일이 '{csv_file_name}'로 저장되었습니다!")

파일이 'tb_user.csv'로 저장되었습니다!


## 'tb_study' : <br>1.스터디 200건 (tb_user에서 선두 200명이 작성한 스터디)

- "study_cd" INT [pk, not null, increment, note: '**스터디 고유코드**']
- "study_title" VARCHAR(200) [not null, note: '**스터디 제목**']
- "study_content" TEXT [not null, note: '**스터디 내용**']
- "user_id" VARCHAR(50) [not null, note: '**작성자 아이디**']
- "study_limit" INT [not null, note: '**모집 인원**']
- "closed_at" TIMESTAMP [not null, note: '**모집 마감날짜**'] <br> **보류**
- "study_status" VARCHAR(10) [not null, note: '**모집 상태**']
- "created_at" TIMESTAMP [not null, default: `CURRENT_TIMESTAMP`, note: '**등록 일자**']
<br> **6개월 전 ~ 현재**

In [134]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 시드 고정
random.seed(42)

# tb_user.csv 파일 읽기
file_path = 'tb_user.csv'
tb_user_df = pd.read_csv(file_path)

# 키워드와 내용 리스트
study_keywords = ["초보자를 위한", "심화 학습", "기초부터 시작하는", "전문가를 위한", "중급자를 위한"]
study_topics = ["파이썬", "머신러닝", "데이터 분석", "인공지능", "웹 개발", "디자인", "수학"]
study_descriptions = [
    "함께 배우는 {topic}의 기초부터 심화까지",
    "이 스터디는 {topic} 학습의 시작점입니다.",
    "함께 {topic}을 깊이 탐구해 봅시다!",
    "초보자도 쉽게 따라올 수 있는 {topic} 스터디입니다.",
    "{topic}에 관심 있는 누구나 참여할 수 있습니다."
]

# tb_study 데이터 생성
study_creators = tb_user_df.head(200)  # 앞의 200명의 유저를 기준으로 스터디 생성
tb_study_data = []

for i, user in enumerate(study_creators.iterrows(), start=1):
    _, user_data = user
    keyword = random.choice(study_keywords)
    topic = random.choice(study_topics)
    description = random.choice(study_descriptions).format(topic=topic)
    
    study_title = f"{keyword} {topic} 스터디"
    study_content = f"{description}\n이 스터디는 {user_data['user_name']}님이 주최했습니다."
    user_id = user_data['user_id']
    study_limit = random.randint(5, 10)  # 5~10명 사이의 모집 인원
    closed_at = (datetime.now() + timedelta(days=random.randint(30, 180))).strftime('%Y-%m-%d %H:%M:%S')  # 30~180일 뒤 마감
    created_at = (datetime.now() - timedelta(days=random.randint(0, 180))).strftime('%Y-%m-%d %H:%M:%S')  # 6개월 전 ~ 현재
    
    tb_study_data.append({
        "study_cd": i,  # 고유 식별자
        "study_title": study_title,
        "study_content": study_content,
        "user_id": user_id,
        "study_limit": study_limit,
        "closed_at": closed_at,
        "study_status": "open",
        "created_at": created_at
    })

# DataFrame으로 변환
tb_study_df = pd.DataFrame(tb_study_data)

# CSV 파일로 저장
tb_study_df.to_csv('tb_study.csv', index=False, encoding='utf-8-sig')
tb_study_df


,study_cd,study_title,study_content,user_id,study_limit,closed_at,study_status,created_at
0,1,초보자를 위한 파이썬 스터디,함께 파이썬을 깊이 탐구해 봅시다!\n이 스터디는 장춘자님이 주최했습니다.,gimyejun@naver.com,6,2025-03-23 16:52:25,open,2024-11-21 16:52:25
1,2,초보자를 위한 디자인 스터디,디자인에 관심 있는 누구나 참여할 수 있습니다.\n이 스터디는 조영호님이 주최했습니다.,coedoyun@daum.net,5,2025-05-13 16:52:25,open,2024-12-18 16:52:25
2,3,초보자를 위한 파이썬 스터디,이 스터디는 파이썬 학습의 시작점입니다.\n이 스터디는 권민지님이 주최했습니다.,lgim@naver.com,6,2025-06-03 16:52:25,open,2024-07-25 16:52:25
3,4,초보자를 위한 웹 개발 스터디,이 스터디는 웹 개발 학습의 시작점입니다.\n이 스터디는 김동현님이 주최했습니다.,ji@hanmail.net,10,2025-06-13 16:52:25,open,2024-09-10 16:52:25
4,5,심화 학습 인공지능 스터디,인공지능에 관심 있는 누구나 참여할 수 있습니다.\n이 스터디는 이유진님이 주최했습니다.,sumin07@naver.com,7,2025-01-26 16:52:25,open,2024-11-16 16:52:25
...,...,...,...,...,...,...,...,...
195,196,기초부터 시작하는 디자인 스터디,초보자도 쉽게 따라올 수 있는 디자인 스터디입니다.\n이 스터디는 장영희님이 주최했...,yi@gmail.com,9,2025-02-03 16:52:25,open,2024-07-21 16:52:25
196,197,초보자를 위한 머신러닝 스터디,함께 머신러닝을 깊이 탐구해 봅시다!\n이 스터디는 최예준님이 주최했습니다.,minjaehwang@naver.com,6,2025-02-17 16:52:25,open,2024-09-06 16:52:25
197,198,초보자를 위한 수학 스터디,함께 배우는 수학의 기초부터 심화까지\n이 스터디는 이진호님이 주최했습니다.,xha@daum.net,8,2025-03-08 16:52:25,open,2024-07-02 16:52:25
198,199,기초부터 시작하는 파이썬 스터디,함께 배우는 파이썬의 기초부터 심화까지\n이 스터디는 이선영님이 주최했습니다.,seomyeongja@daum.net,7,2025-02-08 16:52:25,open,2024-10-12 16:52:25


In [135]:
tb_study_df['study_limit'].sum()

np.int64(1526)

## tb_study_member: <br> 스터디 가입 이력.<br>탈퇴한 이력(status = LEFT)이 1000건 필요함.

-  "member_cd" INT [pk, not null, increment, note: '**스터디 가입 이력 고유 코드**']
- "user_id" VARCHAR(50) [not null, note: '**유저 아이디 (tb_user 참조)**']
- "study_cd" INT [not null, note: '**스터디 고유 코드 (tb_study 참조)**']
- "joined_at" TIMESTAMP [not null, default: `CURRENT_TIMESTAMP`, note: '**가입 일자**'] : <br>**일단 보류**
- "left_at" TIMESTAMP [default: NULL, note: '**탈퇴 일자 (NULL이면 아직 활동 중)**'] : <br>**일단 보류**
- "status" tb_study_member_status_enum [not null, default: 'JOINED', note: '가입 상태 (**JOINED: 가입, LEFT: 탈퇴)**']
}

### 만들 때 필요한 규칙
1. tb_study를 작성한 tb_user(선두 200명)은, 자신이 만든 스터디를 무조건 JOINED:가입한 상태여야 한다.
2. tb_user(끝에서 800명)은 탈퇴한 이력은 0~n 건 이상이다.
3. 하지만 tb_user(끝에서 800명)은 스터디에 가입한 이력(tb_study_member의 status가 "JOINED")은 무조건 있어야 한다.

----------------------------------------------------------------
4. tb_study_member의 status가 "JOINED"인 건수가 tb_study의 study_limit(모집 상한)을 넘겨서는 안됨.

In [136]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 시드 고정
random.seed(42)

# tb_user.csv 및 tb_study.csv 파일 읽기
tb_user_path = 'tb_user.csv'
tb_study_path = 'tb_study.csv'

tb_user_df = pd.read_csv(tb_user_path)
tb_study_df = pd.read_csv(tb_study_path)

# tb_study_member 데이터 생성
tb_study_member_data = []

# Step 1: Ensure all study authors are "JOINED" to their own studies
for study_idx, study in tb_study_df.iterrows():
    tb_study_member_data.append({
        "user_id": study['user_id'],
        "study_cd": study_idx + 1,  # Assuming study_cd corresponds to positional index + 1
        "status": "JOINED",
        "joined_at": (datetime.now() - timedelta(days=random.randint(1, 30))).strftime('%Y-%m-%d %H:%M:%S'),
        "left_at": None
    })

# Step 2: Add participants (last 800 users)
study_limit_tracker = tb_study_df[['study_limit']].copy()
study_limit_tracker['current_count'] = 1  # Initial count includes the author

for user_idx, user in tb_user_df.iloc[200:].iterrows():
    # Ensure the user has at least one JOINED record
    available_studies = study_limit_tracker[study_limit_tracker['current_count'] < study_limit_tracker['study_limit']].index
    if not available_studies.empty:
        joined_study_idx = random.choice(available_studies)
        joined_at = (datetime.now() - timedelta(days=random.randint(1, 30))).strftime('%Y-%m-%d %H:%M:%S')
        tb_study_member_data.append({
            "user_id": user['user_id'],
            "study_cd": joined_study_idx + 1,
            "status": "JOINED",
            "joined_at": joined_at,
            "left_at": None
        })
        study_limit_tracker.loc[joined_study_idx, 'current_count'] += 1

    # Add random LEFT records (0 to n)
    num_left_records = random.randint(0, 5)  # Randomly decide the number of LEFT records
    for _ in range(num_left_records):
        left_available_studies = study_limit_tracker.index
        if not left_available_studies.empty:
            left_study_idx = random.choice(left_available_studies)
            left_joined_at = (datetime.now() - timedelta(days=random.randint(31, 60))).strftime('%Y-%m-%d %H:%M:%S')
            left_left_at = (datetime.strptime(left_joined_at, '%Y-%m-%d %H:%M:%S') + timedelta(days=random.randint(1, 10))).strftime('%Y-%m-%d %H:%M:%S')
            tb_study_member_data.append({
                "user_id": user['user_id'],
                "study_cd": left_study_idx + 1,
                "status": "LEFT",
                "joined_at": left_joined_at,
                "left_at": left_left_at
            })

# Convert to DataFrame
tb_study_member_df = pd.DataFrame(tb_study_member_data)

# Save to CSV
tb_study_member_csv_path = 'tb_study_member.csv'
tb_study_member_df.to_csv(tb_study_member_csv_path, index=False, encoding='utf-8-sig')

print(f"파일이 '{tb_study_member_csv_path}'로 저장되었습니다!")


파일이 'tb_study_member.csv'로 저장되었습니다!


In [137]:
tb_study_member_df['status'].value_counts()

status
LEFT      2095
JOINED    1000
Name: count, dtype: int64

In [138]:
import pandas as pd

# 파일 경로 설정
study_member_path = 'tb_study_member.csv'
user_path = 'tb_user.csv'
study_path = 'tb_study.csv'

# CSV 파일 로드
tb_study_member_df = pd.read_csv(study_member_path)
tb_user_df = pd.read_csv(user_path)
tb_study_df = pd.read_csv(study_path)

# 스터디 모집 상한 정보 설정 (study_cd를 1-based로 조정)
study_limits = tb_study_df.set_index(tb_study_df.index + 1)['study_limit']

# 검증 1: 스터디 작성자가 자신의 스터디에 JOINED 상태인지 확인
def check_authors_joined():
    authors = tb_study_df[['user_id', 'study_title']].set_index('user_id').to_dict()['study_title']
    author_joined_check = tb_study_member_df[tb_study_member_df['status'] == 'JOINED']
    author_in_studies = set(author_joined_check['user_id'].unique())
    missing_authors = set(tb_study_df['user_id']) - author_in_studies
    return missing_authors

# 검증 2: study_limit을 초과하지 않았는지 확인
def check_study_limits():
    joined_counts = tb_study_member_df[tb_study_member_df['status'] == 'JOINED'].groupby('study_cd').size()
    exceeded_studies = joined_counts[joined_counts > study_limits]
    return exceeded_studies

# 검증 3: 마지막 800명의 사용자가 최소 하나의 JOINED 기록을 가지고 있는지 확인
def check_last_800_users():
    last_800_users = tb_user_df.iloc[200:]['user_id']
    last_800_user_joined_check = tb_study_member_df[
        (tb_study_member_df['user_id'].isin(last_800_users)) & (tb_study_member_df['status'] == 'JOINED')
    ]
    users_with_joined = set(last_800_user_joined_check['user_id'].unique())
    missing_joined_users = set(last_800_users) - users_with_joined
    return missing_joined_users

# 검증 4: LEFT 상태의 레코드가 유효한지 확인
def check_left_records():
    left_check = tb_study_member_df[tb_study_member_df['status'] == 'LEFT']
    invalid_left_records = left_check[left_check['left_at'].isnull()]
    return invalid_left_records

# 실행 및 결과 출력
print("검증 1: 작성자의 JOINED 상태 확인")
missing_authors = check_authors_joined()
print(f"Missing Authors: {missing_authors}" if missing_authors else "모든 작성자가 JOINED 상태입니다.")

print("\n검증 2: 스터디 모집 상한 초과 확인")
exceeded_studies = check_study_limits()
print(f"Studies Exceeding Limit:\n{exceeded_studies}" if not exceeded_studies.empty else "모집 상한 초과 없음.")

print("\n검증 3: 마지막 800명의 사용자 JOINED 기록 확인")
missing_joined_users = check_last_800_users()
print(f"Missing JOINED Records for Last 800 Users: {missing_joined_users}" if missing_joined_users else "모든 사용자가 JOINED 상태를 가지고 있습니다.")

print("\n검증 4: LEFT 기록의 유효성 확인")
invalid_left_records = check_left_records()
print(f"Invalid LEFT Records:\n{invalid_left_records}" if not invalid_left_records.empty else "모든 LEFT 상태 기록이 유효합니다.")


검증 1: 작성자의 JOINED 상태 확인
모든 작성자가 JOINED 상태입니다.

검증 2: 스터디 모집 상한 초과 확인
모집 상한 초과 없음.

검증 3: 마지막 800명의 사용자 JOINED 기록 확인
모든 사용자가 JOINED 상태를 가지고 있습니다.

검증 4: LEFT 기록의 유효성 확인
모든 LEFT 상태 기록이 유효합니다.


## tb_keyword : <br>스터디 분위기 키워드

In [139]:
import pandas as pd

# 키워드 데이터
keyword_list = [
    "초보자 친화적", "경험자 중심", "속도감 있는 진행", "여유로운 진행",
    "목표 달성 가능", "체계적인", "열정적인", "편안한", "조용한", "활발한"
]

# tb_keyword 데이터 생성
tb_keyword_data = []
for idx, keyword in enumerate(keyword_list, start=1):
    tb_keyword_data.append({
        "keyword_cd": idx,  # 고유 코드 (자동 증가)
        "keyword_nm": keyword,
        "created_at": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')  # 현재 시간
    })

# DataFrame으로 변환
tb_keyword_df = pd.DataFrame(tb_keyword_data)

# CSV 파일로 저장
tb_keyword_csv_path = 'tb_keyword.csv'
tb_keyword_df.to_csv(tb_keyword_csv_path, index=False, encoding='utf-8-sig')

print(f"tb_keyword 테이블 데이터가 '{tb_keyword_csv_path}'로 저장되었습니다!")

tb_keyword 테이블 데이터가 'tb_keyword.csv'로 저장되었습니다!


## tb_study_review :<br> 스터디에 대한 리뷰 (자신이 소속했던 스터디를 탈퇴한 경우에만...)

- "sview_cd" INT [pk, not null, increment, note: '**리뷰 고유코드**']
- "study_cd" INT [not null, note: '**스터디 고유코드**']
- "sview_keyword" VARCHAR(50) [not null, note: '**리뷰 키워드**']<br> **폐기 컬럼**
- "created_at" TIMESTAMP [not null, default: `CURRENT_TIMESTAMP`, note: '**리뷰 날짜**']
- "user_id" VARCHAR(50) [not null, note: '**사용자 아이디**']
}


In [140]:
import pandas as pd
import random
from datetime import datetime

# 시드 고정
random.seed(42)

# Load your files (adjust paths as necessary)
tb_study_member_df = pd.read_csv('tb_study_member.csv')
tb_user_df = pd.read_csv('tb_user.csv')
tb_keyword_df = pd.read_csv('tb_keyword.csv')
tb_study_df = pd.read_csv('tb_study.csv')

# Filter tb_study_member for LEFT status
left_records = tb_study_member_df[tb_study_member_df['status'] == 'LEFT']

# Prepare the review data without the `sview_keyword` column
tb_study_review_data = []

for _, row in left_records.iterrows():
    tb_study_review_data.append({
        "sview_cd": len(tb_study_review_data) + 1,  # Auto-increment review ID
        "study_cd": row['study_cd'],
        "sview_keyword": None,  # NULL value for the removed column
        "created_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "user_id": row['user_id']
    })

# Convert to DataFrame
tb_study_review_df = pd.DataFrame(tb_study_review_data)

# Save to CSV locally
tb_study_review_df.to_csv('tb_study_review.csv', index=False, encoding='utf-8-sig')

# Display DataFrame info and sample
print(tb_study_review_df.info())
print(tb_study_review_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sview_cd       2095 non-null   int64 
 1   study_cd       2095 non-null   int64 
 2   sview_keyword  0 non-null      object
 3   created_at     2095 non-null   object
 4   user_id        2095 non-null   object
dtypes: int64(2), object(3)
memory usage: 82.0+ KB
None
   sview_cd  study_cd sview_keyword           created_at                user_id
0         1       117          None  2024-12-26 16:52:26   yeweon88@hanmail.net
1         2       130          None  2024-12-26 16:52:26   sanghungim@gmail.com
2         3       130          None  2024-12-26 16:52:26  gyeongsug82@gmail.com
3         4        40          None  2024-12-26 16:52:26  gyeongsug82@gmail.com
4         5       139          None  2024-12-26 16:52:26  gyeongsug82@gmail.com


## tb_study_review_keyword
tb_study_review 테이블과 tb_keyword의 연결 테이블

In [141]:
import pandas as pd
import random

# Set random seed for reproducibility
random.seed(42)

# Load provided files
tb_study_member_path = 'tb_study_member.csv'
tb_study_review_path = 'tb_study_review.csv'
tb_keyword_path = 'tb_keyword.csv'

tb_study_member_df = pd.read_csv(tb_study_member_path)
tb_study_review_df = pd.read_csv(tb_study_review_path)
tb_keyword_df = pd.read_csv(tb_keyword_path)

# Prepare data for tb_study_review_keyword
tb_study_review_keyword_data = []

for _, review in tb_study_review_df.iterrows():
    # Randomly select 3 to 5 unique keywords for the review
    num_keywords = random.randint(3, 5)
    selected_keywords = random.sample(list(tb_keyword_df['keyword_cd']), num_keywords)
    
    for keyword in selected_keywords:
        tb_study_review_keyword_data.append({
            "review_keyword_id": len(tb_study_review_keyword_data) + 1,  # Auto-increment ID
            "sview_cd": review['sview_cd'],
            "keyword_cd": keyword
        })

# Convert to DataFrame
tb_study_review_keyword_df = pd.DataFrame(tb_study_review_keyword_data)

# Save to CSV
output_path = 'tb_study_review_keyword.csv'
tb_study_review_keyword_df.to_csv(output_path, index=False, encoding='utf-8-sig')

# Display structure and sample data
tb_study_review_keyword_df.info(), tb_study_review_keyword_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8448 entries, 0 to 8447
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   review_keyword_id  8448 non-null   int64
 1   sview_cd           8448 non-null   int64
 2   keyword_cd         8448 non-null   int64
dtypes: int64(3)
memory usage: 198.1 KB


(None,
    review_keyword_id  sview_cd  keyword_cd
 0                  1         1           2
 1                  2         1           1
 2                  3         1           5
 3                  4         1          10
 4                  5         1           7)

## 컬럼 확인 코드

In [142]:
import pandas as pd

# 파일 경로를 실제 위치로 수정하세요
tb_study_review_path = 'tb_study_review.csv'
tb_study_review_keyword_path = 'tb_study_review_keyword.csv'
tb_user_path = 'tb_user.csv'
tb_keyword_path = 'tb_keyword.csv'
tb_study_path = 'tb_study.csv'
tb_study_member_path = 'tb_study_member.csv'

# 테이블 로드
tb_study_review_df = pd.read_csv(tb_study_review_path)
tb_study_review_keyword_df = pd.read_csv(tb_study_review_keyword_path)
tb_user_df = pd.read_csv(tb_user_path)
tb_keyword_df = pd.read_csv(tb_keyword_path)
tb_study_df = pd.read_csv(tb_study_path)
tb_study_member_df = pd.read_csv(tb_study_member_path)

# 컬럼명과 데이터 샘플 확인
print("tb_study_review columns:", tb_study_review_df.columns)
print(tb_study_review_df.head(), "\n")

print("tb_study_review_keyword columns:", tb_study_review_keyword_df.columns)
print(tb_study_review_keyword_df.head(), "\n")

print("tb_user columns:", tb_user_df.columns)
print(tb_user_df.head(), "\n")

print("tb_keyword columns:", tb_keyword_df.columns)
print(tb_keyword_df.head(), "\n")

print("tb_study columns:", tb_study_df.columns)
print(tb_study_df.head(), "\n")

print("tb_study_member columns:", tb_study_member_df.columns)
print(tb_study_member_df.head(), "\n")


tb_study_review columns: Index(['sview_cd', 'study_cd', 'sview_keyword', 'created_at', 'user_id'], dtype='object')
   sview_cd  study_cd  sview_keyword           created_at                user_id
0         1       117            NaN  2024-12-26 16:52:26   yeweon88@hanmail.net
1         2       130            NaN  2024-12-26 16:52:26   sanghungim@gmail.com
2         3       130            NaN  2024-12-26 16:52:26  gyeongsug82@gmail.com
3         4        40            NaN  2024-12-26 16:52:26  gyeongsug82@gmail.com
4         5       139            NaN  2024-12-26 16:52:26  gyeongsug82@gmail.com 

tb_study_review_keyword columns: Index(['review_keyword_id', 'sview_cd', 'keyword_cd'], dtype='object')
   review_keyword_id  sview_cd  keyword_cd
0                  1         1           2
1                  2         1           1
2                  3         1           5
3                  4         1          10
4                  5         1           7 

tb_user columns: Index(['user_id'

In [143]:
import pandas as pd

# 파일 경로를 지정합니다
tb_study_review_path = 'tb_study_review.csv'
tb_study_review_keyword_path = 'tb_study_review_keyword.csv'
tb_user_path = 'tb_user.csv'
tb_keyword_path = 'tb_keyword.csv'
tb_study_path = 'tb_study.csv'
tb_study_member_path = 'tb_study_member.csv'

# 데이터 로드
tb_study_review_df = pd.read_csv(tb_study_review_path)
tb_study_review_keyword_df = pd.read_csv(tb_study_review_keyword_path)
tb_user_df = pd.read_csv(tb_user_path)
tb_keyword_df = pd.read_csv(tb_keyword_path)
tb_study_df = pd.read_csv(tb_study_path)
tb_study_member_df = pd.read_csv(tb_study_member_path)

# 검증 수행
validation_results = {}

# 1. Check that all sview_cd in tb_study_review_keyword exist in tb_study_review
validation_results['review_keyword_sview_cd_exists'] = tb_study_review_keyword_df['sview_cd'].isin(tb_study_review_df['sview_cd']).all()

# 2. Check that all keyword_cd in tb_study_review_keyword exist in tb_keyword
validation_results['review_keyword_keyword_cd_exists'] = tb_study_review_keyword_df['keyword_cd'].isin(tb_keyword_df['keyword_cd']).all()

# 3. Check that all study_cd in tb_study_review exist in tb_study
validation_results['review_study_cd_exists'] = tb_study_review_df['study_cd'].isin(tb_study_df['study_cd']).all()

# 4. Check that all user_id in tb_study_review exist in tb_user
validation_results['review_user_id_exists'] = tb_study_review_df['user_id'].isin(tb_user_df['user_id']).all()

# 5. Check that all study_cd in tb_study_member exist in tb_study
validation_results['member_study_cd_exists'] = tb_study_member_df['study_cd'].isin(tb_study_df['study_cd']).all()

# 6. Check that all user_id in tb_study_member exist in tb_user
validation_results['member_user_id_exists'] = tb_study_member_df['user_id'].isin(tb_user_df['user_id']).all()

# 7. Check that all sview_cd in tb_study_review are unique
validation_results['unique_sview_cd_in_review'] = tb_study_review_df['sview_cd'].is_unique

# 검증 결과 출력
print(validation_results)


{'review_keyword_sview_cd_exists': np.True_, 'review_keyword_keyword_cd_exists': np.True_, 'review_study_cd_exists': np.True_, 'review_user_id_exists': np.True_, 'member_study_cd_exists': np.True_, 'member_user_id_exists': np.True_, 'unique_sview_cd_in_review': True}


In [144]:
print(tb_study_df.columns)

Index(['study_cd', 'study_title', 'study_content', 'user_id', 'study_limit',
       'closed_at', 'study_status', 'created_at'],
      dtype='object')


## tb_study_keyword 테이블

In [145]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 시드 고정
random.seed(42)

# tb_study.csv, tb_keyword.csv 읽기
tb_study = pd.read_csv('tb_study.csv')
tb_keyword = pd.read_csv('tb_keyword.csv')

# tb_study_keyword 데이터 생성
tb_study_keyword_data = []
current_time = datetime.now()

for _, study in tb_study.iterrows():
    study_cd = study['study_cd']
    num_keywords = random.randint(3, 5)  # 3~5개의 키워드 선택
    keyword_cds = random.sample(tb_keyword['keyword_cd'].tolist(), num_keywords)
    
    for keyword_cd in keyword_cds:
        tb_study_keyword_data.append({
            "study_cd": study_cd,
            "keyword_cd": keyword_cd,
            "created_at": current_time.strftime('%Y-%m-%d %H:%M:%S')
        })

# DataFrame으로 변환
tb_study_keyword_df = pd.DataFrame(tb_study_keyword_data)

# CSV 파일로 저장
tb_study_keyword_df.to_csv('tb_study_keyword.csv', index=False, encoding='utf-8-sig')

# 결과 확인
tb_study_keyword_df.head()


,study_cd,keyword_cd,created_at
0,1,2,2024-12-26 17:13:18
1,1,1,2024-12-26 17:13:18
2,1,5,2024-12-26 17:13:18
3,1,10,2024-12-26 17:13:18
4,1,7,2024-12-26 17:13:18


## tb_user_keyword 테이블

In [146]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 시드 고정
random.seed(42)

# tb_user.csv, tb_keyword.csv 읽기
tb_user = pd.read_csv('tb_user.csv')
tb_keyword = pd.read_csv('tb_keyword.csv')

# tb_user_keyword 데이터 생성
tb_user_keyword_data = []
current_time = datetime.now()

for _, user in tb_user.iterrows():
    user_id = user['user_id']
    num_keywords = random.randint(3, 5)  # 3~5개의 키워드 선택
    keyword_cds = random.sample(tb_keyword['keyword_cd'].tolist(), num_keywords)
    
    for keyword_cd in keyword_cds:
        tb_user_keyword_data.append({
            "user_id": user_id,
            "keyword_cd": keyword_cd,
            "created_at": current_time.strftime('%Y-%m-%d %H:%M:%S')
        })

# DataFrame으로 변환
tb_user_keyword_df = pd.DataFrame(tb_user_keyword_data)

# CSV 파일로 저장
tb_user_keyword_df.to_csv('tb_user_keyword.csv', index=False, encoding='utf-8-sig')

# 결과 확인
tb_user_keyword_df.head()


,user_id,keyword_cd,created_at
0,gimyejun@naver.com,2,2024-12-26 17:17:28
1,gimyejun@naver.com,1,2024-12-26 17:17:28
2,gimyejun@naver.com,5,2024-12-26 17:17:28
3,gimyejun@naver.com,10,2024-12-26 17:17:28
4,gimyejun@naver.com,7,2024-12-26 17:17:28
